# 优化器类

我们通过张量封装了梯度计算链路，通过层封装了前向传播链路。

现在，我们将引入**优化器**（Optimizer）类来封装反向传播链路，根据梯度更新参数。

-----------------

至此，我们完成了对神经网络模型三条数据链的封装：
* 张量：梯度计算链路；
* 层：前向传播链路；
* 损失函数：前向传播链路和梯度计算链路的连接点；
* 优化器：反向传播链路。

In [61]:
from abc import abstractmethod, ABC
import numpy as np

## 基础架构

### 张量

In [62]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    @property
    def size(self):
        return len(self.data)

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础层

为了配合自动化反向传播的目的，我们需要在层中添加一个新的属性：**参数**（parameters），用来返回本层需要参与反向传播的张量（通常是本层的模型参数）。

In [63]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    @property
    def parameters(self):
        return []

    def __repr__(self):
        return ''

### 基础损失函数

In [64]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

基础优化器是一个抽象类，定义了一个**更新参数**（step）的虚拟接口。

创建一个优化器，我们需要知道：

* parameters：各层参与反向传播的张量列表，这些张量都需要根据梯度进行更新；
* lr：学习率。

In [65]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    @abstractmethod
    def step(self):
        pass

## 数据

### 特征、标签

In [66]:
feature = Tensor([28.1, 58.0])
label = Tensor([165])

## 模型

### 线性层

在线性层，我们把权重和偏置加入参数列表（parameters）。它们需要根据梯度进行更新。

In [67]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad * x.data
            self.bias.grad += np.sum(p.grad, axis=0)

        p.gradient_fn = gradient_fn
        return p

    @property
    def parameters(self):
        return [self.weight, self.bias]

    def __repr__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 损失函数（平均平方差）

In [68]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### 优化器（随机梯度下降）

我们的第一个优化器类是**随机梯度下降优化器**（SGDOptimizer），按照固定的学习率更新参数列表中的所有张量。

In [69]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

## 设置

### 学习率

In [70]:
LEARNING_RATE = 0.00001

## 验证

### 建模

In [71]:
layer = Linear(feature.size, 1)
print(layer)

Linear[weight(1, 2); bias(1,)]


### 推理

In [72]:
prediction = layer(feature)
print(f'prediction: {prediction}')

prediction: Tensor([43.05])


### 评估

In [73]:
loss_fn = MSELoss()
loss = loss_fn(prediction, label)
print(f'loss: {loss}')

loss: Tensor(14871.802500000002)


## 训练

### 梯度计算

In [74]:
loss.backward()

### 反向传播

完成梯度计算以后，我们可以调用优化器的更新参数函数（step），完成一次迭代的反向传播。

完整的计算图通常包括包括：

* 起点：特征值；
* 遍历节点：输入值、中间值，保存在各张量的父节点列表（parents）中；
* 叶节点：模型参数，保存在各层的参数列表（parameters）中；
* 终点：损失值。

其中，由**遍历节点**连接而成的链路是计算图的核心，是梯度计算的遍历路径。而**叶节点**则是反向传播，更新参数的对象。

In [75]:
optimizer = SGDOptimizer(layer.parameters, lr=LEARNING_RATE)
optimizer.step()

### 重新评估

In [76]:
prediction = layer(feature)
loss = loss_fn(prediction, label)
print(f'loss: {loss}')

loss: Tensor(12503.020514375934)


通过优化器类，我们实现了反向传播链路（更新参数）的自动化。